<a href="https://colab.research.google.com/github/Toussaintduk/Renewable_energy_prediction/blob/main/Renewable_energy_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Renewable energy access Analysis
through Dataset from Kaggle
<a href = https://www.kaggle.com/datasets/belayethossainds/renewable-energy-world-wide-19652022> Kaggle dataset

In [4]:
# import necessary Libary
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns


Loading data from Kaggle

In [12]:
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/data/02 modern-renewable-energy-consumption.csv"
df = pd.read_csv(path, encoding='utf-8')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/data/02 modern-renewable-energy-consumption.csv'

In [ ]:
df.head()

Data Preprocessing

In [ ]:
df.info()

## Rename Columns

In [ ]:
df.columns = df.columns.str.replace(' ', '_').str.replace('-', '_').str.lower()
df.columns = df.columns.str.replace('__', '_')
df.columns

In [ ]:
# checking for dimension
print(f"  rows: {df.shape[0]}, columns: {df.shape[1]}")

## cheking The Entity column

In [ ]:
df['entity'].unique()

in these entity there is some leading spaces,some  abbreavaition such as BP  ,
 and some inconsistent names such as world   to handle this use strip to remove the space  and mapping to get clearly entity for further analysis

In [ ]:
df['entity'] = df['entity'].str.strip()

#  Replace inconsistent names
entity_mapping = {
    'Africa (BP)': 'Africa',
    'Asia Pacific (BP)': 'Asia Pacific',
    'Europe (BP)': 'Europe',
    'High-income countries': 'High-Income Countries',
    'Lower-middle-income countries': 'Lower-Middle-Income Countries',
    'Middle Africa (BP)': 'Middle Africa',
    'Middle East (BP)': 'Middle East',
    'Non-OECD (BP)': 'Non-OECD',
    'OECD (BP)': 'OECD',
    'South and Central America (BP)': 'South and Central America',
    'Upper-middle-income countries': 'Upper-Middle-Income Countries',
    'Western Africa (BP)': 'Western Africa',
    'World': 'Global',  # Rename 'World' to 'Global' or filter it out
    'USSR': 'Former Soviet Union'  # Example if needed for specific cases
}

df['entity'] = df['entity'].replace(entity_mapping)

# Step 3: Optionally, filter out rows where 'entity' is not a country or region of interest
# For example, removing 'World' or other aggregate regions
df = df[~df['entity'].isin(['Global', 'World'])]


In [ ]:
df['entity'].unique()

### Checking Missing value

In [ ]:
# checking missing values
missing_val = df.isnull().sum()
missing_perc = df.isnull().sum()/len(df)*100

for col in df.columns:
  if missing_val[col]> 0:
    print(f"{col} have missing Value of {missing_val[col]} at {missing_perc[col]:.2f}")



let's start from Code Column

In [ ]:
# looking at missing code column
df[df['code'].isnull()].head()

In [ ]:
df[df["code"].isnull()]["entity"].unique()

In [ ]:
# Drop missed value of Code since is notcomplicated  to make imputation and don't contibute alot in Further Analysis
df =df.drop("code",axis=1)
# checking again
df.head()

handling Missing value of  geo_biomass_other_twh,solar_generation_twh	wind_generation_twh	hydro_generation_twh

In [ ]:
# since it is few missed value, it can be drop the column

df = df.dropna(subset=['geo_biomass_other_twh','solar_generation_twh','wind_generation_twh','hydro_generation_twh'])

In [ ]:
# to check df again
if df.isnull().sum().max() == 0 :
  print("No missed value !")

In [ ]:
# checking for duplicates
df.duplicated().sum()

Make Total Energy used in year

In [ ]:
df['total_energy'] = df['geo_biomass_other_twh'] + df['solar_generation_twh'] + df['wind_generation_twh'] + df['hydro_generation_twh']
df.head()

## Explatory Data Analysis

In [ ]:
df.describe()

In [ ]:
# distrubution of data  for numerical columns

#  numerical columns
num_columns = df.select_dtypes(include=['float64', 'int64']).columns

# Plot histograms for all numerical columns
df[num_columns].hist(figsize=(12, 10), bins=30, edgecolor='black')
plt.tight_layout()
plt.show()

In [ ]:
# correlation in features
correlation_matrix = df[num_col].corr(method = 'pearson')

correlation_matrix

In [ ]:
# Filter data for years from 2004 onwards
df_filtered = df[df['year'] >= 2004]

# Set up the figure for multiple plots
plt.figure(figsize=(12, 8))

# Define energy columns
energy_columns = ['geo_biomass_other_twh', 'solar_generation_twh', 'wind_generation_twh', 'hydro_generation_twh']

# Plot each energy type across years
for i, col in enumerate(energy_columns, 1):
    plt.subplot(2, 2, i)  # 2x2 grid for 4 energy types
    sns.lineplot(data=df_filtered, x='year', y=col, marker='o')
    plt.title(f'{col} Variance Over Years')
    plt.xlabel('Year')
    plt.ylabel(f'{col}')
    plt.xticks(rotation=45)

    # Ensure the x-axis shows years as integers (not decimals)
    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f'{int(x)}'))

# Add a title for the entire figure
plt.suptitle('Energy Generation Variance Over Years (2004 and Beyond)', fontsize=16, fontweight='bold')

# Adjust layout to make space for the main title
plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout to avoid overlap with title
plt.show()

In [ ]:
entities = df['entity'].unique()  # Get all unique entities

num_entities = len(entities)
nrows = math.ceil(num_entities / 4)  # Calculate number of rows for subplots

# Create the figure and axes
fig, axes = plt.subplots(nrows=nrows, ncols=4, figsize=(20, 5 * nrows), sharex=True)
axes = axes.flatten()

# Define colors for each energy type for consistency
energy_colors = {
    'geo_biomass_other_twh': 'red',
    'solar_generation_twh': 'black',
    'wind_generation_twh': 'blue',
    'hydro_generation_twh': 'green',

}

# Loop through each entity and plot data
for i, entity in enumerate(entities):
    entity_data = df[df['entity'] == entity].sort_values('year')

    # Plot each energy form with corresponding color
    for energy_form, color in energy_colors.items():
        axes[i].plot(entity_data['year'], entity_data[energy_form], color=color, label=energy_form)

    axes[i].set_title(f'Renewable Energy for {entity}', fontsize=12)
    axes[i].set_ylabel('Power Generation (TWh)', fontsize=10)
    axes[i].set_xlabel('Year', fontsize=10)
    axes[i].set_ylim(0, 2000)  # Adjust y-axis range for better visualization
    axes[i].grid(True)

    # Add a legend for each plot
    axes[i].legend(loc='upper left', fontsize=8)

# Remove extra subplots if there are fewer entities than the number of axes
for j in range(len(entities), len(axes)):
    fig.delaxes(axes[j])

# Adjust the layout for better spacing and clarity
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
df['year'].max(),df['year'].min()